In [1]:
!pip install sacrebleu datasets transformers sentencepiece protobuf 



[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python -m pip install --upgrade pip


In [2]:
!pip install safetensors



[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python -m pip install --upgrade pip


# My fine-tuned model vs the baseline MarianMT

In [3]:
import pandas as pd
from datasets import Dataset

In [4]:
# from transformers import MarianMTModel, MarianTokenizer

# # Path to the directory containing the model and config files
# model_path = 'marianmt_fine_tuned11'

# # Load the model and tokenizer
# model = MarianMTModel.from_pretrained(model_path)
# tokenizer = MarianTokenizer.from_pretrained(model_path)

# baseline_model_name = 'Helsinki-NLP/opus-mt-fr-en'
# baseline_model = MarianMTModel.from_pretrained(baseline_model_name)
# baseline_tokenizer = MarianTokenizer.from_pretrained(baseline_model_name)





In [5]:
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast

# Load the model and tokenizer
model = MBartForConditionalGeneration.from_pretrained("facebook/mbart-large-50-many-to-many-mmt")
tokenizer = MBart50TokenizerFast.from_pretrained("facebook/mbart-large-50-many-to-many-mmt")

# Set source and target languages
src_lang = "fr_XX"  # French
tgt_lang = "en_XX"  # English

# Example French text
fr_text = "Bonjour, comment ça va ?"

# Tokenize the input text (French)
tokenizer.src_lang = src_lang
encoded_input = tokenizer(fr_text, return_tensors="pt")

# Generate the translation
generated_tokens = model.generate(**encoded_input, forced_bos_token_id=tokenizer.lang_code_to_id[tgt_lang])

# Decode the translated text (English)
translated_text = tokenizer.decode(generated_tokens[0], skip_special_tokens=True)

# Print the translated text
print(f"Translated Text: {translated_text}")


Translated Text: Hello, how are you?


In [6]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

# Load the tokenizer and model for facebook/m2m100_418M
tokenizer_m2m = AutoTokenizer.from_pretrained("facebook/m2m100_418M")
model_m2m = AutoModelForSeq2SeqLM.from_pretrained("facebook/m2m100_418M")

# Translation function for french to english
# def translate_m2m(text, src_lang="fr", tgt_lang="en"):
#     # Tokenize the input text with specified language pair
#     encoded_input = tokenizer_m2m(text, return_tensors="pt", padding=True)

#     # Generate translation
#     translated = model_m2m.generate(
#         **encoded_input,
#         forced_bos_token_id=tokenizer_m2m.get_lang_id(tgt_lang)
#     )

#     # Decode and return the translated text
#     translated_text = tokenizer_m2m.decode(translated[0], skip_special_tokens=True)
#     return translated_text

# # Example usage
# fr_text = "Bonjour, comment ça va ?"
# translated_text = translate_m2m(fr_text)
# print(f"Translated Text: {translated_text}")  # Should output in English


In [7]:
reference_translations = [
    "She likes to play tennis with her friends in the afternoon.",    # Tatoeba
    "The European Parliament has approved new legislation on climate change.",  # EuroParl
    "I'm just trying to keep my head above water in this city.",    # OpenSubtitles
    "The United States is a leading global power.",    # CCMatrix
    "Scientists have discovered a new species of bird in the Amazon rainforest.",    # News
    "The proposal to increase renewable energy funding has been well received.",    # EuroParl
    "This cake is delicious, and I think you'll love it.",    # Tatoeba
    "The company is expanding its operations to new markets in Asia.",    # CCMatrix
    "They are arguing over the details of the contract.",    # OpenSubtitles
    "The stock market experienced significant volatility due to economic uncertainty.",    # News
    "Global warming has become a major issue for governments worldwide.",    # News
    "You need to make sure you understand the terms before signing the agreement.",    # OpenSubtitles
    "She gave a speech about the importance of education for all children.",    # Tatoeba
    "The team was very excited after winning the championship match.",    # EuroParl
    "The museum has a new exhibit showcasing ancient Roman artifacts.",    # News
    "Many small businesses are struggling to recover from the economic downturn.",    # News
    "The development of artificial intelligence is advancing at an incredible pace.",    # News
    "We will have to work harder to reduce the environmental impact of the industry.",    # EuroParl
    "He has been practicing his speech for days in preparation for the conference.",    # Tatoeba
    "The president will address the nation later this evening.",    # News
    "Electric cars are expected to become more common in the coming years.",    # News
    "Some cities have started investing heavily in smart city technologies.",    # News
    "She managed to finish her project ahead of the deadline.",    # Tatoeba
    "The ambassador met with local leaders to discuss trade agreements.",    # EuroParl
    "The new healthcare law aims to make medical treatment more affordable for citizens.",    # News
    "Environmental groups are calling for stricter regulations on carbon emissions.",    # News
    "We need to reduce our reliance on fossil fuels and focus on renewable energy.",    # EuroParl
    "The teacher encouraged the students to participate in the debate competition.",    # Tatoeba
    "The local government is planning to build a new sports facility in the city.",    # News
    "Scientists are investigating the potential benefits of gene editing.",    # News
    "There are many factors contributing to the rise in global temperatures.",    # News
    "The new law will ensure that companies pay fair wages to their employees.",    # EuroParl
    "She completed her homework and then went out to meet her friends.",    # Tatoeba
    "Public transport in the city is often delayed due to heavy traffic.",    # OpenSubtitles
    "The country is experiencing an economic boom thanks to its booming tech industry.",    # News
    "The mayor spoke about the city's efforts to improve housing affordability.",    # News
    "This painting was created during the Renaissance period.",    # Tatoeba
    "The student council organized a fundraiser for the local animal shelter.",    # Tatoeba
    "After the meeting, they went out for a casual dinner at the new restaurant.",    # OpenSubtitles
    "The earthquake caused widespread damage to buildings and infrastructure.",    # News
    "The climate summit will be held in Paris next year.",    # EuroParl
    "The company is focusing on expanding its presence in Europe and Asia.",    # CCMatrix
    "He helped his colleague finish the report before the deadline.",    # Tatoeba
    "Renewable energy sources, such as solar power, are becoming increasingly popular.",    # News
    "Governments are implementing policies to reduce their carbon footprints.",    # News
    "The airline has introduced new routes to popular vacation destinations.",    # News
    "The president delivered a speech on the importance of international cooperation.",    # EuroParl
    "The local charity is hosting an event to raise funds for underprivileged children.",    # Tatoeba
    "The new book explores the relationship between humans and technology.",    # News
    "A growing number of people are moving to urban areas in search of better job opportunities.",    # News
    "The hotel offers excellent service and beautiful views of the city.",    # OpenSubtitles
    "She found a new job in a different department within the company.",    # Tatoeba
    "The politician promised to address the country's housing crisis.",    # News
    "The president urged people to be patient as the government works to improve the economy.",    # News
    "The healthcare system in many countries is facing significant challenges.",    # News
    "International trade agreements are crucial for maintaining global economic stability.",    # EuroParl
    "He gave an excellent presentation at the conference last week.",    # Tatoeba
    "The university has introduced new programs for international students.",    # News
    "Several companies are working together to develop the next generation of smartphones.",    # News
    "The city is planning to build more parks and green spaces for the public.",    # News
    "The weather forecast predicts sunny skies and mild temperatures for the weekend.",    # OpenSubtitles
    "The government is working on improving access to affordable healthcare.",    # News
    "Several countries are debating the best approach to tackle climate change.",    # EuroParl
    "The factory produces thousands of units every day to meet demand.",    # CCMatrix
    "The new system will streamline the process and make it more efficient.",    # News
    "The company has seen rapid growth over the past few years thanks to its innovative products.",    # News
]

source_sentences = [
    "Elle aime jouer au tennis avec ses amis l'après-midi.",    # Tatoeba
    "Le Parlement européen a approuvé une nouvelle législation sur le changement climatique.",    # EuroParl
    "Je tente juste de garder la tête hors de l'eau dans cette ville.",    # OpenSubtitles
    "Les États-Unis sont une puissance mondiale dominante.",    # CCMatrix
    "Les scientifiques ont découvert une nouvelle espèce d'oiseau dans la forêt amazonienne.",    # News
    "La proposition d'augmenter le financement des énergies renouvelables a été bien reçue.",    # EuroParl
    "Ce gâteau est délicieux, et je pense que tu vas l'adorer.",    # Tatoeba
    "L'entreprise étend ses opérations vers de nouveaux marchés en Asie.",    # CCMatrix
    "Ils se disputent sur les détails du contrat.",    # OpenSubtitles
    "La bourse a connu une grande volatilité en raison de l'incertitude économique.",    # News
    "Le réchauffement climatique est devenu un problème majeur pour les gouvernements du monde entier.",    # News
    "Tu dois t'assurer de comprendre les termes avant de signer l'accord.",    # OpenSubtitles
    "Elle a prononcé un discours sur l'importance de l'éducation pour tous les enfants.",    # Tatoeba
    "L'équipe était très excitée après avoir remporté le match de championnat.",    # EuroParl
    "Le musée a une nouvelle exposition présentant des artefacts romains antiques.",    # News
    "De nombreuses petites entreprises ont du mal à se remettre de la récession économique.",    # News
    "Le développement de l'intelligence artificielle progresse à un rythme incroyable.",    # News
    "Nous devrons travailler plus dur pour réduire l'impact environnemental de l'industrie.",    # EuroParl
    "Il a pratiqué son discours pendant des jours en préparation pour la conférence.",    # Tatoeba
    "Le président s'adressera à la nation ce soir.",    # News
    "Les voitures électriques devraient devenir plus courantes dans les prochaines années.",    # News
    "Certaines villes ont commencé à investir massivement dans les technologies de villes intelligentes.",    # News
    "Elle a réussi à finir son projet avant la date limite.",    # Tatoeba
    "L'ambassadeur a rencontré les dirigeants locaux pour discuter des accords commerciaux.",    # EuroParl
    "La nouvelle loi sur la santé vise à rendre les soins médicaux plus abordables pour les citoyens.",    # News
    "Les groupes environnementaux demandent des régulations plus strictes sur les émissions de carbone.",    # News
    "Nous devons réduire notre dépendance aux combustibles fossiles et nous concentrer sur les énergies renouvelables.",    # EuroParl
    "Le professeur a encouragé les étudiants à participer à la compétition de débat.",    # Tatoeba
    "Le gouvernement local prévoit de construire une nouvelle installation sportive dans la ville.",    # News
    "Les scientifiques enquêtent sur les avantages potentiels de l'édition génétique.",    # News
    "Il existe de nombreux facteurs contribuant à la hausse des températures mondiales.",    # News
    "La nouvelle loi garantira que les entreprises paient des salaires équitables à leurs employés.",    # EuroParl
    "Elle a terminé ses devoirs puis est sortie retrouver ses amis.",    # Tatoeba
    "Les transports publics dans la ville sont souvent retardés à cause de la circulation intense.",    # OpenSubtitles
    "Le pays connaît un boom économique grâce à son industrie technologique florissante.",    # News
    "Le maire a parlé des efforts de la ville pour améliorer l'accessibilité au logement.",    # News
    "Ce tableau a été créé pendant la période de la Renaissance.",    # Tatoeba
    "Le conseil étudiant a organisé une collecte de fonds pour le refuge local pour animaux.",    # Tatoeba
    "Après la réunion, ils sont allés dîner de manière informelle dans le nouveau restaurant.",    # OpenSubtitles
    "Le tremblement de terre a causé des dégâts importants aux bâtiments et aux infrastructures.",    # News
    "Le sommet climatique se tiendra à Paris l'année prochaine.",    # EuroParl
    "L'entreprise se concentre sur l'expansion de sa présence en Europe et en Asie.",    # CCMatrix
    "Il a aidé son collègue à terminer le rapport avant la date limite.",    # Tatoeba
    "Les sources d'énergie renouvelable, telles que l'énergie solaire, sont de plus en plus populaires.",    # News
    "Les gouvernements mettent en place des politiques pour réduire leur empreinte carbone.",    # News
    "La compagnie aérienne a introduit de nouvelles lignes vers des destinations de vacances populaires.",    # News
    "Le président a prononcé un discours sur l'importance de la coopération internationale.",    # EuroParl
    "La charité locale organise un événement pour collecter des fonds pour les enfants défavorisés.",    # Tatoeba
    "Le nouveau livre explore la relation entre l'homme et la technologie.",    # News
    "Un nombre croissant de personnes déménagent dans les zones urbaines à la recherche de meilleures opportunités d'emploi.",    # News
    "L'hôtel offre un excellent service et de belles vues sur la ville.",    # OpenSubtitles
    "Elle a trouvé un nouveau poste dans un autre département au sein de l'entreprise.",    # Tatoeba
    "Le politicien a promis de s'attaquer à la crise du logement du pays.",    # News
    "Le président a exhorté les citoyens à être patients pendant que le gouvernement travaille pour améliorer l'économie.",    # News
    "Le système de santé de nombreux pays fait face à des défis importants.",    # News
    "Les accords commerciaux internationaux sont cruciaux pour maintenir la stabilité économique mondiale.",    # EuroParl
    "Il a fait une excellente présentation lors de la conférence la semaine dernière.",    # Tatoeba
    "L'université a introduit de nouveaux programmes pour les étudiants internationaux.",    # News
    "Plusieurs entreprises collaborent pour développer la prochaine génération de smartphones.",    # News
    "La ville prévoit de construire plus de parcs et d'espaces verts pour le public.",    # News
    "Les prévisions météorologiques annoncent des ciels ensoleillés et des températures modérées pour le week-end.",    # OpenSubtitles
    "Le gouvernement travaille à améliorer l'accès aux soins de santé abordables.",    # News
    "Plusieurs pays débattent de la meilleure approche pour lutter contre le changement climatique.",    # EuroParl
    "L'usine produit des milliers d'unités chaque jour pour répondre à la demande.",    # CCMatrix
    "Le nouveau système simplifiera le processus et le rendra plus efficace.",    # News
    "L'entreprise a connu une croissance rapide ces dernières années grâce à ses produits innovants.",    # News
]

In [8]:
# import torch
# import sacrebleu

# def translate_sentences(sentences, model, tokenizer, batch_size=16):
#     model.eval()
#     device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#     model.to(device)

#     all_translations = []

#     for i in range(0, len(sentences), batch_size):
#         batch = sentences[i:i+batch_size]
#         inputs = tokenizer(batch, return_tensors="pt", padding=True, truncation=True, max_length=512)
#         inputs = {k: v.to(device) for k, v in inputs.items()}

#         with torch.no_grad():
#             translated = model.generate(**inputs, max_length=512)
#         decoded = tokenizer.batch_decode(translated, skip_special_tokens=True)
#         all_translations.extend(decoded)

#         # Free memory
#         del inputs, translated, decoded
#         torch.cuda.empty_cache()

#     return all_translations
    
# all_translations = translate_sentences(source_sentences, model, tokenizer)
# all_translations_custom = translate_sentences(source_sentences, model, tokenizer)

In [9]:
import torch

# Translation function
def translate_sentences_mbart(sentences, model, tokenizer, src_lang="fr_XX", tgt_lang="en_XX", batch_size=16):
    model.eval()
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    all_translations = []

    # Set the source language for the tokenizer
    tokenizer.src_lang = src_lang

    for i in range(0, len(sentences), batch_size):
        batch = sentences[i:i + batch_size]
        
        # Tokenize the batch of sentences (with padding and truncation)
        inputs = tokenizer(batch, return_tensors="pt", padding=True, truncation=True, max_length=512)
        inputs = {k: v.to(device) for k, v in inputs.items()}

        # Generate translations
        with torch.no_grad():
            generated_tokens = model.generate(**inputs, forced_bos_token_id=tokenizer.lang_code_to_id[tgt_lang])
        
        # Decode the generated tokens into readable text
        decoded = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)
        all_translations.extend(decoded)

        # Clear memory (optional)
        del inputs, generated_tokens, decoded
        torch.cuda.empty_cache()

    return all_translations

def translate_m2m_batch(sentences, model, tokenizer, src_lang="fr", tgt_lang="en", batch_size=16):
    model.eval()
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    all_translations = []

    # Get the target language ID
    tgt_lang_id = tokenizer.get_lang_id(tgt_lang)

    for i in range(0, len(sentences), batch_size):
        batch = sentences[i:i + batch_size]

        # Tokenize the batch of sentences
        encoded_input = tokenizer(batch, return_tensors="pt", padding=True, truncation=True)
        encoded_input = {k: v.to(device) for k, v in encoded_input.items()}

        # Generate translations
        with torch.no_grad():
            translated = model.generate(
                **encoded_input,
                forced_bos_token_id=tgt_lang_id
            )

        # Decode the generated translations
        decoded = tokenizer.batch_decode(translated, skip_special_tokens=True)
        all_translations.extend(decoded)

        # Clear memory (optional)
        del encoded_input, translated, decoded
        torch.cuda.empty_cache()

    return all_translations
all_translations = translate_sentences_mbart(source_sentences, model, tokenizer)
all_translations_custom = translate_m2m_batch(source_sentences, model_m2m, tokenizer_m2m)


# Bleu Score
Exact word/phrase matching — how much of the machine translation is in direct alignment with the reference translation.

In [10]:
import sacrebleu

# Flatten the reference translations list (since it's a list of lists)
references = [ref for ref in reference_translations]
    
# Calculate BLEU score for the baseline model
bleu_custom = sacrebleu.corpus_bleu(all_translations_custom, [references], lowercase=True, tokenize='13a')
bleu_baseline = sacrebleu.corpus_bleu(all_translations, [references], lowercase=True, tokenize='13a')

print(f"Baseline BLEU Score: {bleu_baseline.score:.2f}")
print(f"Custom BLEU Score: {bleu_custom.score:.2f}")


Baseline BLEU Score: 68.08
Custom BLEU Score: 25.16


# TER Score
Measures the number of edits (insertions, deletions, substitutions) needed to change the hypothesis into the reference.

In [11]:
ter_custom = sacrebleu.corpus_ter(all_translations_custom, [references])
ter_score = sacrebleu.corpus_ter(all_translations, [references])

print(f"TER for baseline: {ter_score.score:.2f}")
print(f"TER for custom: {ter_custom.score:.2f}")


TER for baseline: 18.51
TER for custom: 71.22


# ChrF score
Works better for languages with complex morphology.

Can detect character-level errors (typos, word segmentation issues).

In [12]:
# ChrF (default settings, good for many languages)
chrf_score = sacrebleu.corpus_chrf(all_translations, [references])
chrf_custom = sacrebleu.corpus_chrf(all_translations_custom, [references])

print(f"ChrF for baseline: {chrf_score.score:.2f}")
print(f"ChrF for custom: {chrf_custom.score:.2f}")
    

ChrF for baseline: 81.79
ChrF for custom: 50.86


# ROUGE-L
Captures content overlap, which makes it suitable for evaluation tasks where fluency and content preservation are crucial.

In [13]:
!pip install rouge-score


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 54.0 MB/s eta 0:00:00
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24988 sha256=ef38fee5440216097a91a260a642a0d2d2495e145c904ff5214594b457999315
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge-score

[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python -m pip install --upgrade pip


In [14]:
from rouge_score import rouge_scorer

# ROUGE-L (using rouge_score package)
scorer = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)
rouge_l_scores = [scorer.score(ref, pred)['rougeL'].fmeasure for ref, pred in zip(references, all_translations)]
avg_rouge_l = 100 * sum(rouge_l_scores) / len(rouge_l_scores)

print(f"Avg ROUGE-L for baseline: {avg_rouge_l:.2f}")

rouge_l_scores_custom = [scorer.score(ref, pred)['rougeL'].fmeasure for ref, pred in zip(references, all_translations_custom)]
avg_rouge_l_for_custom = 100 * sum(rouge_l_scores_custom) / len(rouge_l_scores_custom)

print(f"Avg ROUGE-L for custom: {avg_rouge_l_for_custom:.2f}")

Avg ROUGE-L for baseline: 86.79
Avg ROUGE-L for custom: 45.32


# Error Analysis

In [15]:
for i in range(len(source_sentences)):
    print(f"SRC   : {source_sentences[i]}")
    print(f"REF   : {reference_translations[i]}")
    print(f"BASE  : {all_translations[i]}")
    print(f"CUSTOM: {all_translations_custom[i]}")
    # print(f"Facebook: {all_translations_facebook[i]}")
    print("-" * 60)
    
    # Optional: limit output (e.g., to first 20 examples)
    if i == 19:
        break

SRC   : Elle aime jouer au tennis avec ses amis l'après-midi.
REF   : She likes to play tennis with her friends in the afternoon.
BASE  : She likes to play tennis with her friends in the afternoon.
CUSTOM: She loves to play tennis with her friends the afternoon.
------------------------------------------------------------
SRC   : Le Parlement européen a approuvé une nouvelle législation sur le changement climatique.
REF   : The European Parliament has approved new legislation on climate change.
BASE  : The European Parliament has approved new legislation on climate change.
CUSTOM: The European Parliament approved a new legislation on climate change.
------------------------------------------------------------
SRC   : Je tente juste de garder la tête hors de l'eau dans cette ville.
REF   : I'm just trying to keep my head above water in this city.
BASE  : I'm just trying to keep my head out of the water in this city.
CUSTOM: I try just de garder la tête hors de l'eau dans cette ville.
--

## test_set 2 (newsdiscusstest2015-enfr)

In [16]:
# Read the French file
with open('testing_dataset/newsdiscusstest2015-enfr.fr', 'r', encoding='utf-8') as f_fr:
    french_sentences = [line.strip() for line in f_fr]

# Read the aligned English file
with open('testing_dataset/newsdiscusstest2015-enfr.en', 'r', encoding='utf-8') as f_en:
    english_sentences = [line.strip() for line in f_en]

# Make sure both have the same number of lines
assert len(french_sentences) == len(english_sentences), "Files are not aligned!"

# Create a DataFrame
df = pd.DataFrame({
    'french': french_sentences,
    'english': english_sentences
})

# Optional: Convert to HuggingFace Dataset
dataset = Dataset.from_pandas(df)

# Preview
df

,french,english
0,Les demeurés de UKIP qui refusent ceux qui viv...,This is perfectly illustrated by the UKIP numb...
1,Vous parlez de quand Nigel Farage dit que le N...,You mean Nigel Farage saying the NHS should no...
2,"D'abord vous utilisez des arguments spécieux, ...",You raise a straw man and then knock it down w...
3,Chaque fois que moi ou ma famille avons besoin...,Every time I or my family need to use the NHS ...
4,Je crois que c'est vous qui utilisez des argum...,I think the straw man is yours.
...,...,...
1495,Et si la règle est la même qu'ici l'état est s...,"And if the rule is the same as here, the state..."
1496,Ils vont peut-etre payer les satellites en fré...,They are going to perhaps pay for the satellit...
1497,Cela est d'autant plus fâcheux que de nombreux...,That is all the more regrettable since a lot o...
1498,Au total cela crée une impression erronée cont...,All in all that creates an erroneous impressio...


In [17]:
source_sentences = dataset['french']
reference_translations = dataset['english']


In [18]:
# import torch
# import sacrebleu

# def translate_sentences(sentences, model, tokenizer, batch_size=16):
#     model.eval()
#     device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#     model.to(device)

#     all_translations = []

#     for i in range(0, len(sentences), batch_size):
#         batch = sentences[i:i+batch_size]
#         inputs = tokenizer(batch, return_tensors="pt", padding=True, truncation=True, max_length=512)
#         inputs = {k: v.to(device) for k, v in inputs.items()}

#         with torch.no_grad():
#             translated = model.generate(**inputs, max_length=512)
#         decoded = tokenizer.batch_decode(translated, skip_special_tokens=True)
#         all_translations.extend(decoded)

#         # Free memory
#         del inputs, translated, decoded
#         torch.cuda.empty_cache()

#     return all_translations
    
# all_translations = translate_sentences(source_sentences, baseline_model, baseline_tokenizer)
# all_translations_custom = translate_sentences(source_sentences, model, tokenizer)

In [19]:
import torch

# Translation function
def translate_sentences_mbart(sentences, model, tokenizer, src_lang="fr_XX", tgt_lang="en_XX", batch_size=16):
    model.eval()
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    all_translations = []

    # Set the source language for the tokenizer
    tokenizer.src_lang = src_lang

    for i in range(0, len(sentences), batch_size):
        batch = sentences[i:i + batch_size]
        
        # Tokenize the batch of sentences (with padding and truncation)
        inputs = tokenizer(batch, return_tensors="pt", padding=True, truncation=True, max_length=512)
        inputs = {k: v.to(device) for k, v in inputs.items()}

        # Generate translations
        with torch.no_grad():
            generated_tokens = model.generate(**inputs, forced_bos_token_id=tokenizer.lang_code_to_id[tgt_lang])
        
        # Decode the generated tokens into readable text
        decoded = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)
        all_translations.extend(decoded)

        # Clear memory (optional)
        del inputs, generated_tokens, decoded
        torch.cuda.empty_cache()

    return all_translations

def translate_m2m_batch(sentences, model, tokenizer, src_lang="fr", tgt_lang="en", batch_size=16):
    model.eval()
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    all_translations = []

    # Get the target language ID
    tgt_lang_id = tokenizer.get_lang_id(tgt_lang)

    for i in range(0, len(sentences), batch_size):
        batch = sentences[i:i + batch_size]

        # Tokenize the batch of sentences
        encoded_input = tokenizer(batch, return_tensors="pt", padding=True, truncation=True)
        encoded_input = {k: v.to(device) for k, v in encoded_input.items()}

        # Generate translations
        with torch.no_grad():
            translated = model.generate(
                **encoded_input,
                forced_bos_token_id=tgt_lang_id
            )

        # Decode the generated translations
        decoded = tokenizer.batch_decode(translated, skip_special_tokens=True)
        all_translations.extend(decoded)

        # Clear memory (optional)
        del encoded_input, translated, decoded
        torch.cuda.empty_cache()

    return all_translations
all_translations = translate_sentences_mbart(source_sentences, model, tokenizer)
all_translations_custom = translate_m2m_batch(source_sentences, model_m2m, tokenizer_m2m)


# Bleu Score
Exact word/phrase matching — how much of the machine translation is in direct alignment with the reference translation.

In [20]:
import sacrebleu

# Flatten the reference translations list (since it's a list of lists)
references = [ref for ref in reference_translations]
    
# Calculate BLEU score for the baseline model
bleu_custom = sacrebleu.corpus_bleu(all_translations_custom, [references], lowercase=True, tokenize='13a')
bleu_baseline = sacrebleu.corpus_bleu(all_translations, [references], lowercase=True, tokenize='13a')

print(f"Baseline BLEU Score: {bleu_baseline.score:.2f}")
print(f"Custom BLEU Score: {bleu_custom.score:.2f}")


Baseline BLEU Score: 37.23
Custom BLEU Score: 10.55


# TER Score
Measures the number of edits (insertions, deletions, substitutions) needed to change the hypothesis into the reference.

In [21]:
ter_custom = sacrebleu.corpus_ter(all_translations_custom, [references])
ter_score = sacrebleu.corpus_ter(all_translations, [references])

print(f"TER for baseline: {ter_score.score:.2f}")
print(f"TER for custom: {ter_custom.score:.2f}")


TER for baseline: 50.40
TER for custom: 89.32


# ChrF score
Works better for languages with complex morphology.

Can detect character-level errors (typos, word segmentation issues).

In [22]:
# ChrF (default settings, good for many languages)
chrf_score = sacrebleu.corpus_chrf(all_translations, [references])
chrf_custom = sacrebleu.corpus_chrf(all_translations_custom, [references])

print(f"ChrF for baseline: {chrf_score.score:.2f}")
print(f"ChrF for custom: {chrf_custom.score:.2f}")
    

ChrF for baseline: 59.31
ChrF for custom: 35.24


# ROUGE-L
Captures content overlap, which makes it suitable for evaluation tasks where fluency and content preservation are crucial.

In [23]:
from rouge_score import rouge_scorer

# ROUGE-L (using rouge_score package)
scorer = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)
rouge_l_scores = [scorer.score(ref, pred)['rougeL'].fmeasure for ref, pred in zip(references, all_translations)]
avg_rouge_l = 100 * sum(rouge_l_scores) / len(rouge_l_scores)

print(f"Avg ROUGE-L for baseline: {avg_rouge_l:.2f}")

rouge_l_scores_custom = [scorer.score(ref, pred)['rougeL'].fmeasure for ref, pred in zip(references, all_translations_custom)]
avg_rouge_l_for_custom = 100 * sum(rouge_l_scores_custom) / len(rouge_l_scores_custom)

print(f"Avg ROUGE-L for custom: {avg_rouge_l_for_custom:.2f}")

Avg ROUGE-L for baseline: 63.48
Avg ROUGE-L for custom: 33.24


# Error Analysis

In [24]:
for i in range(len(source_sentences)):
    print(f"SRC   : {source_sentences[i]}")
    print(f"REF   : {reference_translations[i]}")
    print(f"BASE  : {all_translations[i]}")
    print(f"CUSTOM: {all_translations_custom[i]}")
    # print(f"Facebook: {all_translations_facebook[i]}")
    print("-" * 60)
    
    # Optional: limit output (e.g., to first 20 examples)
    if i == 19:
        break

SRC   : Les demeurés de UKIP qui refusent ceux qui vivent avec le VIH sont un exemple parfait.
REF   : This is perfectly illustrated by the UKIP numbties banning people with HIV.
BASE  : The remainder of UKIP who reject those living with HIV are a perfect example.
CUSTOM: The demeurés of UKIP who refuse those who live with HIV are an example perfect.
------------------------------------------------------------
SRC   : Vous parlez de quand Nigel Farage dit que le NHS ne doit pas servir à payer pour ceux qui viennent au Royaume-Uni faire du tourisme de santé, et quand, avec ça à l'esprit, il répond oui quand l'intervieweur lui demande spécifiquement si les personnes vivant avec le VIH sont inclus dans les indésirables.
REF   : You mean Nigel Farage saying the NHS should not be used to pay for people coming to the UK as health tourists, and saying yes when the interviewer specifically asked if, with the aforementioned in mind, people with HIV were included in not being welcome.
BASE  : Yo

## Test_set 3 (news-test2008)


In [25]:
# Read the French file
with open('testing_dataset/news-test2008.fr', 'r', encoding='utf-8') as f_fr:
    french_sentences = [line.strip() for line in f_fr]

# Read the aligned English file
with open('testing_dataset/news-test2008.en', 'r', encoding='utf-8') as f_en:
    english_sentences = [line.strip() for line in f_en]

# Make sure both have the same number of lines
assert len(french_sentences) == len(english_sentences), "Files are not aligned!"

# Create a DataFrame
df = pd.DataFrame({
    'french': french_sentences,
    'english': english_sentences
})

# Optional: Convert to HuggingFace Dataset
dataset = Dataset.from_pandas(df)

# Preview
df

,french,english
0,"L'inflation, en Europe, a dérapé sur l'aliment...",Food: Where European inflation slipped up
1,"L'inflation accélérée, mesurée dans la zone eu...",The skyward zoom in food prices is the dominan...
2,"En novembre, l'augmentation des prix, dans les...",November price hikes were higher than expected...
3,Les prévisions officielles n'ont indiqué que 3...,"Official forecasts predicted just 3 percent, B..."
4,Contrairement aux banques centrales américaine...,"As opposed to the US, UK, and Canadian central..."
...,...,...
2046,Le poker encore difficilement prévisible sur F...,The months-long and at this stage fairly uncle...
2047,Freenet avait à la fin du troisième trimestre ...,"At the close of the third quarter, Freenet had..."
2048,La Holding de United Internet et Drillisch dét...,The holding of United Internet and Drillisch h...
2049,Il a été désormais convenu que Drillisch appor...,"Now, it had been arranged for Drillisch to acq..."


In [26]:
source_sentences = dataset['french']
reference_translations = dataset['english']


In [27]:
# import torch
# import sacrebleu

# def translate_sentences(sentences, model, tokenizer, batch_size=16):
#     model.eval()
#     device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#     model.to(device)

#     all_translations = []

#     for i in range(0, len(sentences), batch_size):
#         batch = sentences[i:i+batch_size]
#         inputs = tokenizer(batch, return_tensors="pt", padding=True, truncation=True, max_length=512)
#         inputs = {k: v.to(device) for k, v in inputs.items()}

#         with torch.no_grad():
#             translated = model.generate(**inputs, max_length=512)
#         decoded = tokenizer.batch_decode(translated, skip_special_tokens=True)
#         all_translations.extend(decoded)

#         # Free memory
#         del inputs, translated, decoded
#         torch.cuda.empty_cache()

#     return all_translations
    
# all_translations = translate_sentences(source_sentences, baseline_model, baseline_tokenizer)
# all_translations_custom = translate_sentences(source_sentences, model, tokenizer)

In [28]:
import torch

# Translation function
def translate_sentences_mbart(sentences, model, tokenizer, src_lang="fr_XX", tgt_lang="en_XX", batch_size=16):
    model.eval()
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    all_translations = []

    # Set the source language for the tokenizer
    tokenizer.src_lang = src_lang

    for i in range(0, len(sentences), batch_size):
        batch = sentences[i:i + batch_size]
        
        # Tokenize the batch of sentences (with padding and truncation)
        inputs = tokenizer(batch, return_tensors="pt", padding=True, truncation=True, max_length=512)
        inputs = {k: v.to(device) for k, v in inputs.items()}

        # Generate translations
        with torch.no_grad():
            generated_tokens = model.generate(**inputs, forced_bos_token_id=tokenizer.lang_code_to_id[tgt_lang])
        
        # Decode the generated tokens into readable text
        decoded = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)
        all_translations.extend(decoded)

        # Clear memory (optional)
        del inputs, generated_tokens, decoded
        torch.cuda.empty_cache()

    return all_translations

def translate_m2m_batch(sentences, model, tokenizer, src_lang="fr", tgt_lang="en", batch_size=16):
    model.eval()
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    all_translations = []

    # Get the target language ID
    tgt_lang_id = tokenizer.get_lang_id(tgt_lang)

    for i in range(0, len(sentences), batch_size):
        batch = sentences[i:i + batch_size]

        # Tokenize the batch of sentences
        encoded_input = tokenizer(batch, return_tensors="pt", padding=True, truncation=True)
        encoded_input = {k: v.to(device) for k, v in encoded_input.items()}

        # Generate translations
        with torch.no_grad():
            translated = model.generate(
                **encoded_input,
                forced_bos_token_id=tgt_lang_id
            )

        # Decode the generated translations
        decoded = tokenizer.batch_decode(translated, skip_special_tokens=True)
        all_translations.extend(decoded)

        # Clear memory (optional)
        del encoded_input, translated, decoded
        torch.cuda.empty_cache()

    return all_translations
all_translations = translate_sentences_mbart(source_sentences, model, tokenizer)
all_translations_custom = translate_m2m_batch(source_sentences, model_m2m, tokenizer_m2m)


# Bleu Score
Exact word/phrase matching — how much of the machine translation is in direct alignment with the reference translation.

In [29]:
import sacrebleu

# Flatten the reference translations list (since it's a list of lists)
references = [ref for ref in reference_translations]
    
# Calculate BLEU score for the baseline model
bleu_custom = sacrebleu.corpus_bleu(all_translations_custom, [references], lowercase=True, tokenize='13a')
bleu_baseline = sacrebleu.corpus_bleu(all_translations, [references], lowercase=True, tokenize='13a')

print(f"Baseline BLEU Score: {bleu_baseline.score:.2f}")
print(f"Custom BLEU Score: {bleu_custom.score:.2f}")


Baseline BLEU Score: 26.49
Custom BLEU Score: 7.45


# TER Score
Measures the number of edits (insertions, deletions, substitutions) needed to change the hypothesis into the reference.

In [30]:
ter_custom = sacrebleu.corpus_ter(all_translations_custom, [references])
ter_score = sacrebleu.corpus_ter(all_translations, [references])

print(f"TER for baseline: {ter_score.score:.2f}")
print(f"TER for custom: {ter_custom.score:.2f}")


TER for baseline: 63.72
TER for custom: 97.96


# ChrF score
Works better for languages with complex morphology.

Can detect character-level errors (typos, word segmentation issues).

In [31]:
# ChrF (default settings, good for many languages)
chrf_score = sacrebleu.corpus_chrf(all_translations, [references])
chrf_custom = sacrebleu.corpus_chrf(all_translations_custom, [references])

print(f"ChrF for baseline: {chrf_score.score:.2f}")
print(f"ChrF for custom: {chrf_custom.score:.2f}")
    

ChrF for baseline: 53.28
ChrF for custom: 34.17


# ROUGE-L
Captures content overlap, which makes it suitable for evaluation tasks where fluency and content preservation are crucial.

In [32]:
from rouge_score import rouge_scorer

# ROUGE-L (using rouge_score package)
scorer = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)
rouge_l_scores = [scorer.score(ref, pred)['rougeL'].fmeasure for ref, pred in zip(references, all_translations)]
avg_rouge_l = 100 * sum(rouge_l_scores) / len(rouge_l_scores)

print(f"Avg ROUGE-L for baseline: {avg_rouge_l:.2f}")

rouge_l_scores_custom = [scorer.score(ref, pred)['rougeL'].fmeasure for ref, pred in zip(references, all_translations_custom)]
avg_rouge_l_for_custom = 100 * sum(rouge_l_scores_custom) / len(rouge_l_scores_custom)

print(f"Avg ROUGE-L for custom: {avg_rouge_l_for_custom:.2f}")

Avg ROUGE-L for baseline: 53.85
Avg ROUGE-L for custom: 25.64


# Error Analysis

In [33]:
for i in range(len(source_sentences)):
    print(f"SRC   : {source_sentences[i]}")
    print(f"REF   : {reference_translations[i]}")
    print(f"BASE  : {all_translations[i]}")
    print(f"CUSTOM: {all_translations_custom[i]}")
    # print(f"Facebook: {all_translations_facebook[i]}")
    print("-" * 60)
    
    # Optional: limit output (e.g., to first 20 examples)
    if i == 19:
        break

SRC   : L'inflation, en Europe, a dérapé sur l'alimentation
REF   : Food: Where European inflation slipped up
BASE  : Inflation in Europe has hit food
CUSTOM: Inflation, in Europe, a depeat on food
------------------------------------------------------------
SRC   : L'inflation accélérée, mesurée dans la zone euro, est due principalement à l'augmentation rapide des prix de l'alimentation.
REF   : The skyward zoom in food prices is the dominant force behind the speed up in eurozone inflation.
BASE  : Increased inflation, measured in the euro area, is mainly due to the rapid rise in food prices.
CUSTOM: The inflation accelerée, meurée dans la zone euro, est due mainly à l'augmentation rapide des prix de l'alimentation.
------------------------------------------------------------
SRC   : En novembre, l'augmentation des prix, dans les 13 pays de la zone euro, a été plus importante par rapport aux prévisions, après un taux d'inflation de 2,6 pour cent en octobre, une inflation annuelle de 3

## Test_set 4 (newstest2009)

In [34]:
# Read the French file
with open('testing_dataset/newstest2009.fr', 'r', encoding='utf-8') as f_fr:
    french_sentences = [line.strip() for line in f_fr]

# Read the aligned English file
with open('testing_dataset/newstest2009.en', 'r', encoding='utf-8') as f_en:
    english_sentences = [line.strip() for line in f_en]

# Make sure both have the same number of lines
assert len(french_sentences) == len(english_sentences), "Files are not aligned!"

# Create a DataFrame
df = pd.DataFrame({
    'french': french_sentences,
    'english': english_sentences
})

# Optional: Convert to HuggingFace Dataset
dataset = Dataset.from_pandas(df)

# Preview
df

,french,english
0,"Au terme des échanges, la bourse de Prague bas...",Prague Stock Market falls to minus by the end ...
1,"Après la chute de mardi matin, la bourse de Pr...","After a sharp drop in the morning, the Prague ..."
2,Les transactions avec les actions ČEZ atteigni...,Transactions with stocks from the Czech Energy...
3,"Mardi, dès le début des échanges, la bourse de...",The Prague Stock Market immediately continued ...
4,"Cette fois-ci, la baisse est due à la chute de...",This time the fall in stocks on Wall Street is...
...,...,...
2520,Verdant a annoncé avoir une longue liste de cl...,Verdant has said that it has a long list of cl...
2521,"Après les vagues, les marées.","After the waves, the tides."
2522,"Un intérêt particulier, disent-ils à l'Institu...","Particular interest, declare the people at the..."
2523,Bien qu'il s'agisse d'un phénomène intermitten...,"Even though it is an intermittent phenomenon, ..."


In [35]:
source_sentences = dataset['french']
reference_translations = dataset['english']


In [36]:
# import torch
# import sacrebleu

# def translate_sentences(sentences, model, tokenizer, batch_size=16):
#     model.eval()
#     device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#     model.to(device)

#     all_translations = []

#     for i in range(0, len(sentences), batch_size):
#         batch = sentences[i:i+batch_size]
#         inputs = tokenizer(batch, return_tensors="pt", padding=True, truncation=True, max_length=512)
#         inputs = {k: v.to(device) for k, v in inputs.items()}

#         with torch.no_grad():
#             translated = model.generate(**inputs, max_length=512)
#         decoded = tokenizer.batch_decode(translated, skip_special_tokens=True)
#         all_translations.extend(decoded)

#         # Free memory
#         del inputs, translated, decoded
#         torch.cuda.empty_cache()

#     return all_translations
    
# all_translations = translate_sentences(source_sentences, baseline_model, baseline_tokenizer)
# all_translations_custom = translate_sentences(source_sentences, model, tokenizer)

In [37]:
import torch

# Translation function
def translate_sentences_mbart(sentences, model, tokenizer, src_lang="fr_XX", tgt_lang="en_XX", batch_size=16):
    model.eval()
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    all_translations = []

    # Set the source language for the tokenizer
    tokenizer.src_lang = src_lang

    for i in range(0, len(sentences), batch_size):
        batch = sentences[i:i + batch_size]
        
        # Tokenize the batch of sentences (with padding and truncation)
        inputs = tokenizer(batch, return_tensors="pt", padding=True, truncation=True, max_length=512)
        inputs = {k: v.to(device) for k, v in inputs.items()}

        # Generate translations
        with torch.no_grad():
            generated_tokens = model.generate(**inputs, forced_bos_token_id=tokenizer.lang_code_to_id[tgt_lang])
        
        # Decode the generated tokens into readable text
        decoded = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)
        all_translations.extend(decoded)

        # Clear memory (optional)
        del inputs, generated_tokens, decoded
        torch.cuda.empty_cache()

    return all_translations

def translate_m2m_batch(sentences, model, tokenizer, src_lang="fr", tgt_lang="en", batch_size=16):
    model.eval()
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    all_translations = []

    # Get the target language ID
    tgt_lang_id = tokenizer.get_lang_id(tgt_lang)

    for i in range(0, len(sentences), batch_size):
        batch = sentences[i:i + batch_size]

        # Tokenize the batch of sentences
        encoded_input = tokenizer(batch, return_tensors="pt", padding=True, truncation=True)
        encoded_input = {k: v.to(device) for k, v in encoded_input.items()}

        # Generate translations
        with torch.no_grad():
            translated = model.generate(
                **encoded_input,
                forced_bos_token_id=tgt_lang_id
            )

        # Decode the generated translations
        decoded = tokenizer.batch_decode(translated, skip_special_tokens=True)
        all_translations.extend(decoded)

        # Clear memory (optional)
        del encoded_input, translated, decoded
        torch.cuda.empty_cache()

    return all_translations
all_translations = translate_sentences_mbart(source_sentences, model, tokenizer)
all_translations_custom = translate_m2m_batch(source_sentences, model_m2m, tokenizer_m2m)


# Bleu Score
Exact word/phrase matching — how much of the machine translation is in direct alignment with the reference translation.

In [38]:
import sacrebleu

# Flatten the reference translations list (since it's a list of lists)
references = [ref for ref in reference_translations]
    
# Calculate BLEU score for the baseline model
bleu_custom = sacrebleu.corpus_bleu(all_translations_custom, [references], lowercase=True, tokenize='13a')
bleu_baseline = sacrebleu.corpus_bleu(all_translations, [references], lowercase=True, tokenize='13a')

print(f"Baseline BLEU Score: {bleu_baseline.score:.2f}")
print(f"Custom BLEU Score: {bleu_custom.score:.2f}")


Baseline BLEU Score: 30.30
Custom BLEU Score: 8.14


# TER Score
Measures the number of edits (insertions, deletions, substitutions) needed to change the hypothesis into the reference.

In [39]:
ter_custom = sacrebleu.corpus_ter(all_translations_custom, [references])
ter_score = sacrebleu.corpus_ter(all_translations, [references])

print(f"TER for baseline: {ter_score.score:.2f}")
print(f"TER for custom: {ter_custom.score:.2f}")


TER for baseline: 58.73
TER for custom: 94.97


# ChrF score
Works better for languages with complex morphology.

Can detect character-level errors (typos, word segmentation issues).

In [40]:
# ChrF (default settings, good for many languages)
chrf_score = sacrebleu.corpus_chrf(all_translations, [references])
chrf_custom = sacrebleu.corpus_chrf(all_translations_custom, [references])

print(f"ChrF for baseline: {chrf_score.score:.2f}")
print(f"ChrF for custom: {chrf_custom.score:.2f}")
    

ChrF for baseline: 55.83
ChrF for custom: 34.65


# ROUGE-L
Captures content overlap, which makes it suitable for evaluation tasks where fluency and content preservation are crucial.

In [41]:
from rouge_score import rouge_scorer

# ROUGE-L (using rouge_score package)
scorer = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)
rouge_l_scores = [scorer.score(ref, pred)['rougeL'].fmeasure for ref, pred in zip(references, all_translations)]
avg_rouge_l = 100 * sum(rouge_l_scores) / len(rouge_l_scores)

print(f"Avg ROUGE-L for baseline: {avg_rouge_l:.2f}")

rouge_l_scores_custom = [scorer.score(ref, pred)['rougeL'].fmeasure for ref, pred in zip(references, all_translations_custom)]
avg_rouge_l_for_custom = 100 * sum(rouge_l_scores_custom) / len(rouge_l_scores_custom)

print(f"Avg ROUGE-L for custom: {avg_rouge_l_for_custom:.2f}")

Avg ROUGE-L for baseline: 57.49
Avg ROUGE-L for custom: 25.65


# Error Analysis

In [42]:
for i in range(len(source_sentences)):
    print(f"SRC   : {source_sentences[i]}")
    print(f"REF   : {reference_translations[i]}")
    print(f"BASE  : {all_translations[i]}")
    print(f"CUSTOM: {all_translations_custom[i]}")
    # print(f"Facebook: {all_translations_facebook[i]}")
    print("-" * 60)
    
    # Optional: limit output (e.g., to first 20 examples)
    if i == 19:
        break

SRC   : Au terme des échanges, la bourse de Prague bascula dans le négatif
REF   : Prague Stock Market falls to minus by the end of the trading day
BASE  : At the end of the exchanges, the Prague Stock Exchange is down in the negative
CUSTOM: Au terme des échanges, la bourse de Prague bascula dans le négatif
------------------------------------------------------------
SRC   : Après la chute de mardi matin, la bourse de Prague réctifia ses pertes.
REF   : After a sharp drop in the morning, the Prague Stock Market corrected its losses.
BASE  : After the fall of Tuesday morning, the Prague Stock Exchange corrected its losses.
CUSTOM: After the fall of Tuesday morning, the bourse of Prague rectifia ses pertes.
------------------------------------------------------------
SRC   : Les transactions avec les actions ČEZ atteignirent presque la moitié des échanges quotidiens habituels.
REF   : Transactions with stocks from the Czech Energy Enterprise (ČEZ) reached nearly half of the regular dail

## Test_set 5 (newstest2014-fren)

In [43]:
# Read the French file
with open('testing_dataset/newstest2014-fren.fr', 'r', encoding='utf-8') as f_fr:
    french_sentences = [line.strip() for line in f_fr]

# Read the aligned English file
with open('testing_dataset/newstest2014-fren.en', 'r', encoding='utf-8') as f_en:
    english_sentences = [line.strip() for line in f_en]

# Make sure both have the same number of lines
assert len(french_sentences) == len(english_sentences), "Files are not aligned!"

# Create a DataFrame
df = pd.DataFrame({
    'french': french_sentences,
    'english': english_sentences
})

# Optional: Convert to HuggingFace Dataset
dataset = Dataset.from_pandas(df)

# Preview
df

,french,english
0,"Spectaculaire saut en ""wingsuit"" au-dessus de ...",Spectacular Wingsuit Jump Over Bogota
1,Le sportif Jhonathan Florez a sauté jeudi d'un...,Sportsman Jhonathan Florez jumped from a helic...
2,Equipé d'un wingsuit (une combinaison munie d'...,"Wearing a wingsuit, he flew past over the famo..."
3,Une boîte noire dans votre voiture ?,A black box in your car?
4,Alors que les planificateurs du réseau routier...,As America's road planners struggle to find th...
...,...,...
2998,La commission scolaire Marguerite-Bourgeoys a ...,The Marguerite-Bourgeoys School Board has crea...
2999,"Rachida Azdouz, de l'Université de Montréal, e...",Rachida Azdouz from the University of Montreal...
3000,La préparation à gérer une classe dans un cont...,Preparation to manage a class in a North-Ameri...
3001,"""Des stratégies pédagogiques différentes, c'es...","""The real need is for different educational st..."


In [44]:
source_sentences = dataset['french']
reference_translations = dataset['english']


In [45]:
# import torch
# import sacrebleu

# def translate_sentences(sentences, model, tokenizer, batch_size=16):
#     model.eval()
#     device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#     model.to(device)

#     all_translations = []

#     for i in range(0, len(sentences), batch_size):
#         batch = sentences[i:i+batch_size]
#         inputs = tokenizer(batch, return_tensors="pt", padding=True, truncation=True, max_length=512)
#         inputs = {k: v.to(device) for k, v in inputs.items()}

#         with torch.no_grad():
#             translated = model.generate(**inputs, max_length=512)
#         decoded = tokenizer.batch_decode(translated, skip_special_tokens=True)
#         all_translations.extend(decoded)

#         # Free memory
#         del inputs, translated, decoded
#         torch.cuda.empty_cache()

#     return all_translations
    
# all_translations = translate_sentences(source_sentences, baseline_model, baseline_tokenizer)
# all_translations_custom = translate_sentences(source_sentences, model, tokenizer)

In [46]:
import torch

# Translation function
def translate_sentences_mbart(sentences, model, tokenizer, src_lang="fr_XX", tgt_lang="en_XX", batch_size=16):
    model.eval()
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    all_translations = []

    # Set the source language for the tokenizer
    tokenizer.src_lang = src_lang

    for i in range(0, len(sentences), batch_size):
        batch = sentences[i:i + batch_size]
        
        # Tokenize the batch of sentences (with padding and truncation)
        inputs = tokenizer(batch, return_tensors="pt", padding=True, truncation=True, max_length=512)
        inputs = {k: v.to(device) for k, v in inputs.items()}

        # Generate translations
        with torch.no_grad():
            generated_tokens = model.generate(**inputs, forced_bos_token_id=tokenizer.lang_code_to_id[tgt_lang])
        
        # Decode the generated tokens into readable text
        decoded = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)
        all_translations.extend(decoded)

        # Clear memory (optional)
        del inputs, generated_tokens, decoded
        torch.cuda.empty_cache()

    return all_translations

def translate_m2m_batch(sentences, model, tokenizer, src_lang="fr", tgt_lang="en", batch_size=16):
    model.eval()
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    all_translations = []

    # Get the target language ID
    tgt_lang_id = tokenizer.get_lang_id(tgt_lang)

    for i in range(0, len(sentences), batch_size):
        batch = sentences[i:i + batch_size]

        # Tokenize the batch of sentences
        encoded_input = tokenizer(batch, return_tensors="pt", padding=True, truncation=True)
        encoded_input = {k: v.to(device) for k, v in encoded_input.items()}

        # Generate translations
        with torch.no_grad():
            translated = model.generate(
                **encoded_input,
                forced_bos_token_id=tgt_lang_id
            )

        # Decode the generated translations
        decoded = tokenizer.batch_decode(translated, skip_special_tokens=True)
        all_translations.extend(decoded)

        # Clear memory (optional)
        del encoded_input, translated, decoded
        torch.cuda.empty_cache()

    return all_translations
all_translations = translate_sentences_mbart(source_sentences, model, tokenizer)
all_translations_custom = translate_m2m_batch(source_sentences, model_m2m, tokenizer_m2m)


# Bleu Score
Exact word/phrase matching — how much of the machine translation is in direct alignment with the reference translation.

In [47]:
import sacrebleu

# Flatten the reference translations list (since it's a list of lists)
references = [ref for ref in reference_translations]
    
# Calculate BLEU score for the baseline model
bleu_custom = sacrebleu.corpus_bleu(all_translations_custom, [references], lowercase=True, tokenize='13a')
bleu_baseline = sacrebleu.corpus_bleu(all_translations, [references], lowercase=True, tokenize='13a')

print(f"Baseline BLEU Score: {bleu_baseline.score:.2f}")
print(f"Custom BLEU Score: {bleu_custom.score:.2f}")


Baseline BLEU Score: 37.79
Custom BLEU Score: 8.37


# TER Score
Measures the number of edits (insertions, deletions, substitutions) needed to change the hypothesis into the reference.

In [48]:
ter_custom = sacrebleu.corpus_ter(all_translations_custom, [references])
ter_score = sacrebleu.corpus_ter(all_translations, [references])

print(f"TER for baseline: {ter_score.score:.2f}")
print(f"TER for custom: {ter_custom.score:.2f}")


TER for baseline: 49.47
TER for custom: 96.79


# ChrF score
Works better for languages with complex morphology.

Can detect character-level errors (typos, word segmentation issues).

In [49]:
# ChrF (default settings, good for many languages)
chrf_score = sacrebleu.corpus_chrf(all_translations, [references])
chrf_custom = sacrebleu.corpus_chrf(all_translations_custom, [references])

print(f"ChrF for baseline: {chrf_score.score:.2f}")
print(f"ChrF for custom: {chrf_custom.score:.2f}")
    

ChrF for baseline: 61.95
ChrF for custom: 35.75


# ROUGE-L
Captures content overlap, which makes it suitable for evaluation tasks where fluency and content preservation are crucial.

In [50]:
from rouge_score import rouge_scorer

# ROUGE-L (using rouge_score package)
scorer = rouge_scorer.RougeScorer(['rougeL'], use_stemmer=True)
rouge_l_scores = [scorer.score(ref, pred)['rougeL'].fmeasure for ref, pred in zip(references, all_translations)]
avg_rouge_l = 100 * sum(rouge_l_scores) / len(rouge_l_scores)

print(f"Avg ROUGE-L for baseline: {avg_rouge_l:.2f}")

rouge_l_scores_custom = [scorer.score(ref, pred)['rougeL'].fmeasure for ref, pred in zip(references, all_translations_custom)]
avg_rouge_l_for_custom = 100 * sum(rouge_l_scores_custom) / len(rouge_l_scores_custom)

print(f"Avg ROUGE-L for custom: {avg_rouge_l_for_custom:.2f}")

Avg ROUGE-L for baseline: 64.88
Avg ROUGE-L for custom: 27.76


# Error Analysis

In [51]:
for i in range(len(source_sentences)):
    print(f"SRC   : {source_sentences[i]}")
    print(f"REF   : {reference_translations[i]}")
    print(f"BASE  : {all_translations[i]}")
    print(f"CUSTOM: {all_translations_custom[i]}")
    # print(f"Facebook: {all_translations_facebook[i]}")
    print("-" * 60)
    
    # Optional: limit output (e.g., to first 20 examples)
    if i == 19:
        break

SRC   : Spectaculaire saut en "wingsuit" au-dessus de Bogota
REF   : Spectacular Wingsuit Jump Over Bogota
BASE  : Spectacular "wingsuit" jump over Bogota
CUSTOM: Spectaculaire saut in "wingsuit" au-dessus de Bogota
------------------------------------------------------------
SRC   : Le sportif Jhonathan Florez a sauté jeudi d'un hélicoptère au-dessus de Bogota, la capitale colombienne.
REF   : Sportsman Jhonathan Florez jumped from a helicopter above Bogota, the capital of Colombia, on Thursday.
BASE  : Athlete Jhonathan Florez jumped from a helicopter over Bogota, Colombia's capital, Thursday.
CUSTOM: Jhonathan Florez a sauté jeudi d'un hélicoptère au-dessus de Bogota, la capitale colombienne.
------------------------------------------------------------
SRC   : Equipé d'un wingsuit (une combinaison munie d'ailes), il est passé à 160 km/h au-dessus du célèbre sanctuaire Monserrate, situé à plus de 3 000 mètres d'altitude, où de nombreux badauds s'étaient rassemblés pour observer son e